In [1]:
import sys
import os

# Get the project directory
project_path = "/home/diego-ngz/Git/thesis-tabtrans"

sys.path.append(project_path) #This helps to be able to import the data from the parent directory to other files

from utils import tabtrans_file, data
import numpy as np

/home/diego-ngz/Git/thesis-tabtrans


In [2]:
id = 1484

X_train, X_test, y_train, y_test, train_indices, val_indices, n_instances, n_labels, n_numerical, n_categories = data.import_data(id)

INFO:openml.datasets.dataset:pickle write lsvt


In [10]:
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)

X_train shape:  (100, 310)
X_test shape:  (26, 310)


In [3]:
from sklearn.model_selection import StratifiedKFold

# Initialize StratifiedKFold with 5 splits
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 11)



In [18]:
skf.split(X_train, y_train)

dict_split = {}
i = 1


for x,y in skf.split(X_train, y_train):

    dict_split[f"split_{i}"] = set(y)    
    i += 1

In [ ]:
'''
n_layers_lst = [2, 3, 4, 5] #2, 3, 4, 5
n_heads_lst = [4, 8, 16, 32] #4, 8, 16, 32
embed_dim = [128,256] #The embedding size is set one by one to avoid the out of memory error {128, 256}
batch_size = 32 # 32, 64, 128, 256, 512, 1024
epochs = 100
sample_size = [100,80,60,40,20]
'''
n_layers = 2
n_heads = 4
embedding_size = 128
batch_size = 32
epochs = 50


model, metrics = tabtrans_file.general_tabtrans(X_train, X_test, y_train, y_test, train_indices, val_indices, n_labels, n_numerical, n_categories, n_layers, n_heads, embedding_size, batch_size, epochs)


In [ ]:
metrics